# Development Tools

Python provides modules which help us in writing softwares:
- `pydoc` for documentation
- `doctest` & `unittest` for unit tests
- `typing` for type hints

## 1. Typing

`typing` module provides runtime support for type hints in various scenarios.

### 1.1. Type aliases

A type alias is defined by assigning type to the alias.

In [2]:
Vector = list[float]

def alpha(scalar: float, vector: Vector, built: str) -> Vector:
    # do something
    return vector

In above case  `Vector` and `list[float]` will be treated as interchangeable synonyms.

Type aliases are useful for simplifying the complex type signatures.
```
fom collections.abc import Sequence

ConnectionOptions = dict[str, str]
Address = tuple[str, int]

Server = tuple[Address, ConnectionOptions]

def broadcast_message(message: str, servers: Sequence[Server])-> None:
     # some code


# This above mentioned function without Type alias would look like this

def broadcast_message(message: str, servers: Sequence[tuple[tuple[str, int], dict[str, str]]]) -> None:
    # some code
```

Type aliases may be marked with `TypeAlias` to make it explicit that the statement is a type alias declaration, not a normal variable assignment.

```
from typing import TypeAlias

Vector: TypeAlias = list[float]
```

### 1.2. NewType

New type create distinct types.

```
from typing import NewType

UserId = NewType('UserId', int)

id1 = UserId(111111)
```

The static type checker will treat the new type as if it were a subclass of the original type. This is useful in helping catch logical errors:

```
def get_user_name(user_id: UserId) -> str:
    ...

# passes type checking
user_a = get_user_name(UserId(42351))

# fails type checking; an int is not a UserId
user_b = get_user_name(-1)
```

We may still perform all `int` operations on a variable of type `UserId`, but the result will always be of type `int`. This lets you pass in a `UserId` wherever an `int` might be expected, but will prevent you from accidentally creating a `UserId` in an invalid way.

```
# 'output' is of type 'int', not 'UserId'
output = UserId(23413) + UserId(54341)
```

These checks are enforced only by the static type checker. At runtime, the statement `Derived = NewType('Derived', Base)` will make `Derived` a callable that immediately returns whatever parameter you pass it. That means the expression `Derived(some_value)` does not create a new class or introduce much overhead beyond that of a regular function call.

More precisely, the expression `some_value is Derived(some_value)` is always true at runtime.

It is invalid to create a subtype of Derived:

```
from typing import NewType

UserId = NewType('UserId', int)

# Fails at runtime and does not pass type checking
class AdminUserId(UserId): pass
```

However, it is possible to create a NewType based on a ‘derived’ NewType:

```
from typing import NewType

UserId = NewType('UserId', int)

ProUserId = NewType('ProUserId', UserId)
```

### 1.3. Annotating Callable Objects

All callable objects can annotated using `collections.abc.Callable` or `typing.Callable.Callable[[argument list], return variable]` .

Ex - `.Callable[[int], str]`signifies a function that takes a single parameter of type `int` and returns a `str`.

In [3]:
from collections.abc import Callable, Awaitable

def feeder(get_next_item: Callable[[], str]) -> None:
    ...  # Body

def async_query(on_success: Callable[[int], None],
                on_error: Callable[[int, Exception], None]) -> None:
    pass

async def on_update(value: str) -> None:
    pass

callback: Callable[[str], Awaitable[None]] = on_update

The argument list must be a list of types. The return type must be a single type.

If a literal ellipsis `...` is given as the argument list, it indicates that a callable with any arbitrary parameter list would be acceptable.

In [4]:
def concat(x: str, y: str)->str:
    return x + y

x: Callable[..., str]
x = str
x = concat

`Callable` can't express complex signatures, so to do that we define these complex signatures in `__call__()` method of a custom class which inherits from the `Protocol` class.

In [10]:
from collections.abc import Iterable
from typing import Protocol

class Combiner(Protocol):
    def __call__(self, *vals: bytes, maxlen: int ) -> list[bytes]: ...

def batch_proc(data: Iterable[bytes], cb_results: Combiner) -> bytes:
    for item in data:
        ...

def good_cb(*vals: bytes, maxlen: int ) -> list[bytes]:
    ...
def bad_cb(*vals: bytes, maxitems: int) -> list[bytes]:
    ...

batch_proc([], good_cb)  

### 1.4. Generics

Type information about objects kept in containers cannot be inferred in a generic way.

Python provides some container classes that help use to extract those information.

Here we can observe that `Sequence[Car]` indicates that all elements in the sequence must be instance of `Car` class

In [ ]:
from collections.abc import Sequence

class Car: ...

def car_data(car: Sequence[Car], name: str)-> None:...

We can make annotations to objects without using a class also.

In [12]:
from typing import TypeVar
from collections.abc import Sequence

C = TypeVar('C')

def car_data(car: Sequence[C], name: str)-> None:...

### 1.5. Any Type

`Any` is a special kind of type.

A static type checker will treat ever type as being compatible with `Any` and `Any` as being compatible with every type.

Thus it is possible to perform any operation or method call on a value of type `Any` and assign it to any variable.

In [13]:
from typing import Any

# declaring Any type variable
a: Any = None

# Any can be any type of variable 
a = []

a = ""

a = 10

a = True

In [14]:
# A function can also accept argument of Any type

def func(a: Any)-> None:
    ...

In [16]:
type(a) # latest assigned type is being given to a 

bool

## 1.6. Classes, Functions in Typing

`typing` module has various classes, functions and decorators.


**Classes in Typing**
- `typing.AnyStr` it is used for functions that may accept `str` or `bytes` arguments but can't allow to mix them
  - `AnyStr = TypeVar('AnyStr', str, bytes)`

- `typing.LiteralString` special type that only includes literal string 
  - useful for where we generate tokens or passkeys made up of literals only and are strings, so it makes possible to avoid any no literal string generation by acting as static checker


- `typing.Never` a type that has no members, it can be used to define a function that should never be called, or a function that never returns

Only python 3.11+

In [ ]:
from typing import Never

def alpha(arg: Never)->None:
    ...

- `typing.NoReturn` special type to indicate that function never returns

In [20]:
from typing import NoReturn

def alpha()->NoReturn:
    ...

- `typing.Self` special type to represent current enclosed class

Only python 3.11+

In [ ]:
from typing import Self

class Beta:
    def return_Self(self)-> Self:
        return self
    

class Car:
    def return_car(car)-> Car:
        ...

- `typing.TypeAlias` used to annotate aliases that make use of forward reference
- `typing.Optional` `Optional[X]` is equivalent to `X|None` or `Union[X, None]`

Only supported over 3.10+

In [22]:
from typing import Optional

def func(arg: Optional[int] = None)-> None:
    ...

- `typing.Concatenate` special form for annotating higher order functions
- `typing.Literal` special form for annotating literal types
- `typing.ClassVar` special type construct to mark class variables

In [23]:
from typing import ClassVar

class Robot:
    type: ClassVar[str]
    def __init__(self, ammo, hud) -> None:
        self.ammo = ammo

# ammo is instance variable it changes for each instance while hud remains same for all instances throughout the class

- `typing.Final` special type to indicate final names tp type checkers

In [24]:
from typing import Final

HEIGHT_WIN: Final = 720

class Layout:
    Dimension: Final[tuple] = (1080, 720)

- `typing.Required` special type construct to mark a `TypedDict` as required
- `typing.NotRequired` special type construct to mark a `TypedDict` keys as potentially missing
- `typing.Annotated` special type construct to add context-specific metadata to an annotation
    - like this: `Annotated[type, metadata]`

In [27]:
from typing import Annotated
from dataclasses import dataclass

@dataclass
class Values:
    low: int
    high: int

T1 = Annotated[int, Values(0, 10)]
T1 = Annotated[int, Values(100, 1000)]

- `typing.TypeGuard` special type construct for making user-defined type guard functions
  - it can be used to annotate the return type of user defined type guard function
  - it only accepts a single type argument 
  - it aims to benefit type narrowing

In [29]:
def is_str(val: str or float):
    # isinstance a type guard
    if isinstance(val, str):
        # type of val is narrowed to str
        ...
    else: 
        # type of val is narrowed to float
        ...

In [ ]:
# using typeguard

from typing import TypeGuard

def is_str_list(val: list[object]) -> TypeGuard[list[str]]:
    '''Determines whether all objects in the list are strings'''
    return all(isinstance(x, str) for x in val)

- `typing.Unpack` special constructor to mark an object as being unpacked
- `typing.Generic` abstract base class for generic type
- `typing.TypeVar` type variable
- `typing.TypeVarTuple` type variable tuple
- `typing.PramSpec` parameter specification variable
- `typing.PramSpecArgs` parameter specification of arguments
- `typing.PramSpecKwargs` parameter specification of keyword arguments
- `typing.NamedTuple` `typing` version of `collections.namedTuple()`
- `typing.NewType` class to create lower head
- `typing.Protocol` base class for Protocol
- `typing.TypedDict` special construct to add type hints to a dictionary

**Functions in Typing**
- `typing.cast(typ, val)` cast a value to a type
- `typing.assert_type(val, typ. /)` ask a static type checker to confirm that `val` has an inferred type of `typ`
- `typing.assert_never(arg, /)` ask a static type checker to confirm that a line of code is unreachable
- `typing.reveal_type(obj, /)` reveal the inferred static type of an expression

## 2. Pydoc

`pydoc` module automatically generates documentation from python modules.

This documentation can be presented as:
- text on console
- save as HTML pages
- served to browser

For any module, class, functions, attributes documentation is derived from the docstring `__doc__` attribute of the object and recursively of its documentable members.

In case their is no docstring, `pydoc` tries to obtain description from the block of the comment lines just above the definition of the class, function or method source file or at the top of the module.

The built-in function help() invokes the online help system in the interactive interpreter, which uses `pydoc` to generate its documentation as text on the console.

To view a list of all possible options associated with `pydoc` and generate documentation use this command:
``` 
python -m pydoc
```

In [32]:
__doc__

'Automatically created module for IPython interactive environment'

In [34]:
!python -m pydoc

pydoc - the Python documentation tool

pydoc <name> ...
    Show text documentation on something.  <name> may be the name of a
    Python keyword, topic, function, module, or package, or a dotted
    reference to a class or function within a module or module in a
    package.  If <name> contains a '\', it is used as the path to a
    Python source file to document. If name is 'keywords', 'topics',
    or 'modules', a listing of these things is displayed.

pydoc -k <keyword>
    Search for a keyword in the synopsis lines of all available modules.

pydoc -n <hostname>
    Start an HTTP server with the given hostname (default: localhost).

pydoc -p <port>
    Start an HTTP server on the given port on the local machine.  Port
    number 0 can be used to get an arbitrary unused port.

pydoc -b
    Start an HTTP server on an arbitrary unused port and open a Web browser
    to interactively browse documentation.  This option can be used in
    combination with -n and/or -p.

pydoc -w <name> .

This will load all documentation of `module` in command prompt

In [ ]:
!python -m pydoc numpy

To generate documentation about custom documentation use
```
python -m pydoc -w my_module
```